# (0) Install dependencies for golang.

In [ ]:
# (1.1) run this cell first time using python runtime
!apt update
!apt install golang-go
!apt-get install libdw-dev
%env GOPATH=/root/go
# (1.2) then refresh, it will now use gophernotes. Skip to golang in later cells

#(1)Install BMF

In [ ]:
!pip install BabitMF


#(2) Configure the environment variable to reference the BMF C library with go.

In [ ]:
! which bmf_env
! cat /usr/local/bin/bmf_env
! bmf_env

Copy and execute it in your terminal, in colab, we use %env:

In [ ]:
%env C_INCLUDE_PATH=/usr/local/lib/python3.10/dist-packages/bmf/include
%env CPLUS_INCLUDE_PATH=/usr/local/lib/python3.10/dist-packages/bmf/include
%env LIBRARY_PATH=/usr/local/cuda/lib64/stubs:/usr/local/lib/python3.10/dist-packages/bmf/lib
%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib/python3.10/dist-packages/bmf/lib

#(3) Download a sample transcoded video and mov from our assets (using Big Bunny as an example here).

In [ ]:
!wget https://github.com/BabitMF/bmf/releases/download/files/files.tar.gz
!tar -zvxf files.tar.gz

# (4) Implement a simple transcoding function With BMF Go SDK

In [ ]:
%%writefile /content/sync_mode.go
package main
import (
  "github.com/babitmf/bmf-gosdk/bmf"
  "fmt"
  "os"
	"time"
)
func syncModeSerial() {
	decoder, err0 := bmf.NewModuleFunctorBuiltin("c_ffmpeg_decoder", map[string]interface{}{
		"input_path": "./files/big_bunny_10s_30fps.mp4",
	}, 0, 1)
	defer decoder.Free()
	if decoder == nil {
			fmt.Printf("Load decoder module failed %v\n", err0)
	} else {
			fmt.Printf("Load decoder module successful\n")
	}

	encoder, err1 := bmf.NewModuleFunctorBuiltin("c_ffmpeg_encoder", map[string]interface{}{
		"output_path": "./output_sync_mode_serial.mp4",
	}, 1, 0)
	defer encoder.Free()
	if encoder == nil {
			fmt.Printf("Load encoder module failed %v\n", err1)
	} else {
			fmt.Printf("Load encoder module successful\n")
	}

	pass, err2 := bmf.NewModuleFunctorBuiltin("pass_through", nil, 1, 1)
	defer pass.Free()
	if pass == nil {
			fmt.Printf("Load pass_through module failed %v\n", err2)
	} else {
			fmt.Printf("Load pass_through module successful\n")
	}
	fmt.Println("Sync mode Serial")

	for {
		// do decoder module
        is_done, err := decoder.Execute([]*bmf.Packet{}, true)
        if is_done {
			fmt.Printf("Decode done\n")

			// EOF packet
			p_eof := bmf.GenerateEofPacket()
			pkts_eof := []*bmf.Packet{}
			pkts_eof = append(pkts_eof, p_eof)
			is_done_eof, err_eof := pass.Execute(pkts_eof, true)
			if is_done_eof {
					break
			}
			if err_eof != nil {
				fmt.Println("PassThrough EOF execute failed error : %v\n", err_eof)
			}
			opkts_eof, err1_eof := pass.Fetch(0)
			if err1_eof != nil {
				fmt.Println("PassThrough EOF fetch failed error : %v\n", err1_eof)
			}

			for i := 0; i < len(opkts_eof); i++ {
				_, err_enc_eof := encoder.Call([]*bmf.Packet{opkts_eof[i]})
				if err_enc_eof != nil {
					fmt.Printf("encoder EOF call failed! error : %v\n",err_enc_eof)
				}
			}
			break
        }

        if err != nil {
			fmt.Printf("decoder execute failed! error : %v\n", err)
			break
        }

        opkts_dec, err1 := decoder.Fetch(0)
        if err1 != nil {
			fmt.Printf("decoder fetch failed!\n, error : %v", err1)
            break
        }

		// do pass_through module
		for i := 0; i < len(opkts_dec); i++ {
			ipkts_pass := []*bmf.Packet{}
			ipkts_pass = append(ipkts_pass, opkts_dec[i])
			_, err2 := pass.Execute(ipkts_pass, true)
			if err2 != nil {
				fmt.Printf("Pass Through failed! %v\n", err2)
				break
			}

			opkts_pass, err1 := pass.Fetch(0)
			if err1 != nil {
				fmt.Printf("Pass Through Fetch failed! error : %v\n", err1)
				break
			}

			// do encoder module
			for i := 0; i < len(opkts_pass); i++ {
				_, err := encoder.Call([]*bmf.Packet{opkts_pass[i]})
				if err != nil {
					fmt.Printf("encoder call failed! error : %v\n", err)
				}
			}

		 }
	}
}

func syncMode() {
	done := false
	decoder, err0 := bmf.NewModuleFunctorBuiltin("c_ffmpeg_decoder", map[string]interface{}{
		"input_path": "./files/big_bunny_10s_30fps.mp4",
	}, 0, 1)
	if decoder == nil {
			fmt.Printf("Load decoder module failed %v\n", err0)
	} else {
			fmt.Printf("Load decoder module successful\n")
	}

	encoder, err1 := bmf.NewModuleFunctorBuiltin("c_ffmpeg_encoder", map[string]interface{}{
		"output_path": "./output_sync_mode.mp4",
	}, 1, 0)
	if encoder == nil {
			fmt.Printf("Load encoder module failed %v\n", err1)
	} else {
			fmt.Printf("Load encoder module successful\n")
	}

	pass, err2 := bmf.NewModuleFunctorBuiltin("pass_through", nil, 1, 1)
	if pass == nil {
			fmt.Printf("Load pass_through module failed %v\n", err2)
	} else {
			fmt.Printf("Load pass_through module successful\n")
	}
	passChan := make(chan []*bmf.Packet, 10)
	encChan := make(chan []*bmf.Packet, 10)
	fmt.Println("Sync mode")
	// PassThrough
	go func() {
		for opkts_dec := range passChan {
			eofSet := false

			// do pass_through module
			for i := 0; i < len(opkts_dec); i++ {
				ipkts_pass := []*bmf.Packet{}
				ipkts_pass = append(ipkts_pass, opkts_dec[i])
				if opkts_dec[i].Timestamp() == bmf.EOF {
					eofSet = true
				}
				_, err2 := pass.Execute(ipkts_pass, true)
				if err2 != nil {
					fmt.Printf("Pass Through failed! %v\n", err2)
					break
				}

				opkts_pass, err1 := pass.Fetch(0)
				if err1 != nil {
					fmt.Printf("Pass Through Fetch failed! error : %v\n", err1)
					break
				}
				// do encoder module
				for i := 0; i < len(opkts_pass); i++ {
					encChan <- []*bmf.Packet{opkts_pass[i]}
				}

			}

			if eofSet {
				fmt.Println("pass done")
				encChan <- []*bmf.Packet{bmf.GenerateEofPacket()}
				pass.Free()
				break
			}
		}
	}()

		// Encoder
	go func() {
		for opkts_pass := range encChan {
			eofSet := false

			// do encoder module
			for i := 0; i < len(opkts_pass); i++ {
				_, err := encoder.Call([]*bmf.Packet{opkts_pass[i]})
				if err != nil {
					fmt.Printf("encoder call failed! error : %v\n", err)
					break
				}
				if opkts_pass[i].Timestamp() == bmf.EOF {
					eofSet = true
				}
			}

			if eofSet {
				fmt.Println("enc done")
				encoder.Free()
				done = true
				break
			}
		}
	}()

	for {
		// do decoder module
        is_done, err := decoder.Execute([]*bmf.Packet{}, true)
        if is_done {
			fmt.Printf("Decode done\n")

			// EOF packet
			p_eof := bmf.GenerateEofPacket()
			pkts_eof := []*bmf.Packet{}
			pkts_eof = append(pkts_eof, p_eof)
			passChan <- pkts_eof
			decoder.Free()
			break
        }

        if err != nil {
			fmt.Printf("decoder execute failed! error : %v\n", err)
			break
        }

        opkts_dec, err1 := decoder.Fetch(0)
        if err1 != nil {
			fmt.Printf("decoder fetch failed!\n, error : %v", err1)
            break
        }

		// do pass_through module
		for i := 0; i < len(opkts_dec); i++ {
			ipkts_pass := []*bmf.Packet{}
			ipkts_pass = append(ipkts_pass, opkts_dec[i])
			passChan <- ipkts_pass
		 }
	}

	for !done {
		time.Sleep(100 * time.Millisecond)
	}

}

func main() {
	if os.Args[1] == "syncMode" {
	    syncMode()
	}
	if os.Args[1] == "syncModeSerial" {
	    syncModeSerial()
	}
}

#(6) Compile the code and call the bmf library through go.

In [ ]:
!go mod init test
!go mod tidy
!go build sync_mode.go

In [ ]:
!./sync_mode syncModeSerial

In [ ]:
!./sync_mode syncMode

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 800):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return f"""
  <video width={video_width} controls>
    <source src="{video_url}">
  </video>
  """

video_url1 = show_video('./files/big_bunny_10s_30fps.mp4')
video_url2 = show_video('output_sync_mode_serial.mp4')
video_url3 = show_video('output_sync_mode.mp4')

html = video_url1 + video_url2 + video_url3
HTML(html)